In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as mt
import matplotlib.pyplot as plt
import os

# from scipy.stats import kendalltau
# from scipy.stats import spearmanr

In [2]:
label = pd.read_csv("/mnt/yzy/NIMCGCN/datasets/data(MDA108)/m-d.csv",delimiter=',',header=None).values

In [3]:
def AU_ROC(label, pred):
    fpr, tpr, thresholds = mt.roc_curve(label, pred, pos_label=1)
    roc_auc = mt.auc(fpr, tpr)

    best_idx = np.argmax(tpr - fpr)
    best_threshold = thresholds[best_idx]
    # print(best_threshold)
    return roc_auc, best_threshold

def AU_PRC(label, pred):
    precision, recall, thresholds = mt.precision_recall_curve(label, pred)
    prc_auc = mt.auc(recall, precision)
    return prc_auc

def MRR(label, pred):
    mrr = mt.label_ranking_average_precision_score([label],[pred])
    return mrr

def HIT(label, pred, K):
    sorted_indices = np.argsort(pred)[::-1]
    hits = label[sorted_indices[:K]].sum()
    return hits/K

def MSE(label, pred):
    mse = mt.mean_squared_error(label, pred)
    return mse

# def CI(label, pred):
#     CI_k, pv_k = kendalltau(label, pred)
#     CI_s, pv_s = spearmanr(label, pred)    
#     return pv_k, pv_s

In [7]:
# 单个或者多个结果分析
count = 0
label_1d = label.reshape(-1)
'''key: name of the model'''
'''vakue: prediction score'''
scores = {"label":label_1d}

scores["Ystar"] = np.load("Prediction/Ablation/MultiViewSAGE_Yupdated_10FoldCV_230520_[lr]0.0005_[wd]0.0_[ep]100_[cvMthd]elem_[miRDim]64_[drugDim]64_[kFdim]64_[alpha]0.3_[loss]WMSE.npy")
scores["Y"] = np.load("Prediction/Ablation/MultiViewSAGE_Y_10FoldCV_230520_[lr]0.0005_[wd]0.0_[ep]100_[cvMthd]elem_[miRDim]64_[drugDim]64_[kFdim]64_[alpha]0.3_[loss]WMSE.npy")

for name,pred in scores.items():
    count += 1
    pred_1d = pred.reshape(-1)
    
    if pred_1d.shape[0] != label_1d.shape[0]:
        print("Error: {} and {} have different shape".format(dir, name))
        continue

    '''基于打分的指标'''
    '''计算AUC'''
    auc,threshold = AU_ROC(label_1d, pred_1d)
    auc = 1 - auc if auc < 0.5 else auc

    '''计算AU_PRC'''
    aupr = AU_PRC(label_1d, pred_1d)

    '''计算MRR'''
    mrr = MRR(label_1d, pred_1d)

    '''计算HIT'''
    hit_50  = HIT(label_1d, pred_1d, 50)
    hit_100 = HIT(label_1d, pred_1d, 100)

    '''计算MSE'''
    mse = MSE(label_1d, pred_1d)

    '''基于标签的指标'''
    pred_y = (pred_1d >= threshold).astype(int)
    '''分类指标'''
    acc = mt.accuracy_score(label_1d, pred_y)
    recall = mt.recall_score(label_1d, pred_y)
    precision = mt.precision_score(label_1d, pred_y,zero_division=0)
    f1 = mt.f1_score(label_1d, pred_y)


    exp_method = name.split("_")[0]
    # print("{}/{}: AUC = {:.8f}, AUPR = {:.8f}, MRR = {:.8f}, HIT = {:.8f}, ACC = ".format(dir, exp_method, auc, aupr, mrr, hit))
    print("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}"\
            .format(name, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))

print(count)

label     /label          :	 AUC =  1.00000, AUPR = 1.00000, MRR = 1.00000, HIT@50 = 1.00000, HIT@100 = 1.00000, MSE = 0.00000, ACC = 1.00000, RECALL = 1.00000, PRECISION = 1.00000, F1 = 1.00000
Ystar     /Ystar          :	 AUC =  0.76752, AUPR = 0.00389, MRR = 0.00393, HIT@50 = 0.00000, HIT@100 = 0.05000, MSE = 0.00106, ACC = 0.71788, RECALL = 0.69231, PRECISION = 0.00155, F1 = 0.00310
Y         /Y              :	 AUC =  0.62745, AUPR = 0.00135, MRR = 0.00137, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.00080, ACC = 0.69605, RECALL = 0.49580, PRECISION = 0.00103, F1 = 0.00206
3


In [12]:
# 所有结果联合分析 并写入文件
# output = open("./metric_output_5f_@50@100.txt", "w+")

count = 0
label_1d = label.reshape(-1)
for dir in os.listdir("Prediction/"):
    if dir != "Ablation":
        continue
    for file in os.listdir("Prediction/{}".format(dir)):
        count += 1

        pred = np.load("Prediction/{}/{}".format(dir,file))
        pred_1d = pred.reshape(-1)
        
        if pred_1d.shape[0] != label_1d.shape[0]:
            print("Error: {} and {} have different shape".format(dir, file))
            continue

        '''基于打分的指标'''
        '''计算AUC'''
        auc,threshold = AU_ROC(label_1d, pred_1d)
        auc = 1 - auc if auc < 0.5 else auc

        '''计算AU_PRC'''
        aupr = AU_PRC(label_1d, pred_1d)

        '''计算MRR'''
        mrr = MRR(label_1d, pred_1d)

        '''计算HIT'''
        hit_50  = HIT(label_1d, pred_1d, 50)
        hit_100 = HIT(label_1d, pred_1d, 100)

        '''计算MSE'''
        mse = MSE(label_1d, pred_1d)

        '''基于标签的指标'''
        pred_y = (pred_1d >= threshold).astype(int)
        '''分类指标'''
        acc = mt.accuracy_score(label_1d, pred_y)
        recall = mt.recall_score(label_1d, pred_y)
        precision = mt.precision_score(label_1d, pred_y,zero_division=0)
        f1 = mt.f1_score(label_1d, pred_y)


        exp_method = file.split("_")[0]+"_"+file.split("_")[-1].split(']')[-1].split('.')[0][0:5]
        # print("{}/{}: AUC = {:.8f}, AUPR = {:.8f}, MRR = {:.8f}, HIT = {:.8f}, ACC = ".format(dir, exp_method, auc, aupr, mrr, hit))
        print("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}"\
              .format(dir, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))
        # output.write("{:10}/{:15}:\t AUC =  {:.5f}, AUPR = {:.5f}, MRR = {:.5f}, HIT@50 = {:.5f}, HIT@100 = {:.5f}, MSE = {:.5f}, ACC = {:.5f}, RECALL = {:.5f}, PRECISION = {:.5f}, F1 = {:.5f}\n"\
        #       .format(dir, exp_method, auc, aupr, mrr, hit_50, hit_100, mse, acc, recall, precision, f1))

# output.close()
print(count)

Ablation  /MultiViewSAGE_WMSE:	 AUC =  0.62745, AUPR = 0.00135, MRR = 0.00137, HIT@50 = 0.00000, HIT@100 = 0.00000, MSE = 0.00080, ACC = 0.69605, RECALL = 0.49580, PRECISION = 0.00103, F1 = 0.00206
Ablation  /MultiViewSAGE_WMSE:	 AUC =  0.76752, AUPR = 0.00389, MRR = 0.00393, HIT@50 = 0.00000, HIT@100 = 0.05000, MSE = 0.00106, ACC = 0.71788, RECALL = 0.69231, PRECISION = 0.00155, F1 = 0.00310
Ablation  /MultiViewGCN_WMSE:	 AUC =  0.81450, AUPR = 0.01035, MRR = 0.01063, HIT@50 = 0.08000, HIT@100 = 0.12000, MSE = 0.00063, ACC = 0.73620, RECALL = 0.76224, PRECISION = 0.00183, F1 = 0.00364
Ablation  /MultiViewGCN_CONTR:	 AUC =  0.81189, AUPR = 0.00943, MRR = 0.00965, HIT@50 = 0.10000, HIT@100 = 0.13000, MSE = 0.00063, ACC = 0.75933, RECALL = 0.73986, PRECISION = 0.00194, F1 = 0.00388
4
